In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from osgeo import gdal
from PIL import Image as im

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
# for hyperparameter tuning 
from sklearn.model_selection import train_test_split, GridSearchCV,RepeatedStratifiedKFold
# for model evaluation 
from sklearn.metrics import mean_squared_error,r2_score
import random



2023-01-27 15:07:08.369835: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-27 15:07:08.441711: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-27 15:07:08.441729: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-27 15:07:08.855089: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
df=pd.read_csv("features_metadata_FzP19JI.csv")
df.head()

,filename,chip_id,satellite,split,month,size,cksum,s3path_us,s3path_eu,s3path_as,corresponding_agbm
0,0003d2eb_S1_00.tif,0003d2eb,S1,train,September,1049524,3953454613,s3://drivendata-competition-biomassters-public...,s3://drivendata-competition-biomassters-public...,s3://drivendata-competition-biomassters-public...,0003d2eb_agbm.tif
1,0003d2eb_S1_01.tif,0003d2eb,S1,train,October,1049524,3531005382,s3://drivendata-competition-biomassters-public...,s3://drivendata-competition-biomassters-public...,s3://drivendata-competition-biomassters-public...,0003d2eb_agbm.tif
2,0003d2eb_S1_02.tif,0003d2eb,S1,train,November,1049524,1401197002,s3://drivendata-competition-biomassters-public...,s3://drivendata-competition-biomassters-public...,s3://drivendata-competition-biomassters-public...,0003d2eb_agbm.tif
3,0003d2eb_S1_03.tif,0003d2eb,S1,train,December,1049524,3253084255,s3://drivendata-competition-biomassters-public...,s3://drivendata-competition-biomassters-public...,s3://drivendata-competition-biomassters-public...,0003d2eb_agbm.tif
4,0003d2eb_S1_04.tif,0003d2eb,S1,train,January,1049524,2467836265,s3://drivendata-competition-biomassters-public...,s3://drivendata-competition-biomassters-public...,s3://drivendata-competition-biomassters-public...,0003d2eb_agbm.tif


In [3]:
dataset=df[["filename","split","corresponding_agbm","month","chip_id","satellite"]]
dataset.head()


,filename,split,corresponding_agbm,month,chip_id,satellite
0,0003d2eb_S1_00.tif,train,0003d2eb_agbm.tif,September,0003d2eb,S1
1,0003d2eb_S1_01.tif,train,0003d2eb_agbm.tif,October,0003d2eb,S1
2,0003d2eb_S1_02.tif,train,0003d2eb_agbm.tif,November,0003d2eb,S1
3,0003d2eb_S1_03.tif,train,0003d2eb_agbm.tif,December,0003d2eb,S1
4,0003d2eb_S1_04.tif,train,0003d2eb_agbm.tif,January,0003d2eb,S1


In [4]:
data=dataset[(dataset["month"]=="June")  | (dataset["month"]=="July") | (dataset["month"]=="August") | (dataset["month"]=="September")]

In [5]:
data=data[data["split"]=="train"]
data

,filename,split,corresponding_agbm,month,chip_id,satellite
0,0003d2eb_S1_00.tif,train,0003d2eb_agbm.tif,September,0003d2eb,S1
9,0003d2eb_S1_09.tif,train,0003d2eb_agbm.tif,June,0003d2eb,S1
10,0003d2eb_S1_10.tif,train,0003d2eb_agbm.tif,July,0003d2eb,S1
11,0003d2eb_S1_11.tif,train,0003d2eb_agbm.tif,August,0003d2eb,S1
12,0003d2eb_S2_00.tif,train,0003d2eb_agbm.tif,September,0003d2eb,S2
...,...,...,...,...,...,...
252391,fff05995_S1_11.tif,train,fff05995_agbm.tif,August,fff05995,S1
252392,fff05995_S2_00.tif,train,fff05995_agbm.tif,September,fff05995,S2
252400,fff05995_S2_09.tif,train,fff05995_agbm.tif,June,fff05995,S2
252401,fff05995_S2_10.tif,train,fff05995_agbm.tif,July,fff05995,S2


In [6]:
cchip=data["chip_id"].unique()
data["chip_id"].nunique()
sam=random.sample(list(cchip),240)
len(sam)

240

In [7]:
k="0003d2eb"

In [8]:
onechip=data[data["chip_id"]==k]
onechip

,filename,split,corresponding_agbm,month,chip_id,satellite
0,0003d2eb_S1_00.tif,train,0003d2eb_agbm.tif,September,0003d2eb,S1
9,0003d2eb_S1_09.tif,train,0003d2eb_agbm.tif,June,0003d2eb,S1
10,0003d2eb_S1_10.tif,train,0003d2eb_agbm.tif,July,0003d2eb,S1
11,0003d2eb_S1_11.tif,train,0003d2eb_agbm.tif,August,0003d2eb,S1
12,0003d2eb_S2_00.tif,train,0003d2eb_agbm.tif,September,0003d2eb,S2
18,0003d2eb_S2_09.tif,train,0003d2eb_agbm.tif,June,0003d2eb,S2
19,0003d2eb_S2_10.tif,train,0003d2eb_agbm.tif,July,0003d2eb,S2
20,0003d2eb_S2_11.tif,train,0003d2eb_agbm.tif,August,0003d2eb,S2


In [9]:
learnrate=[]
n_esti=[]
score=[]

for chip in sam:
    onechip=data[data["chip_id"]==chip]
    image_numpy={}
    for i in range(len(onechip)):
        image_name=onechip["filename"].iloc[i]
        fileLocation="trainFeatures/"+image_name
        ds=gdal.Open(fileLocation)
        data_array= ds.GetRasterBand(1).ReadAsArray()
        data_array=np.array(data_array)
        data_array=data_array.reshape(-1)
        col_name = onechip['satellite'].iloc[i] + '_' + onechip["month"].iloc[i]
        image_numpy[col_name] = data_array
#   adding Agbm part to list to make it a dictionary.
    agbm_filename = onechip['corresponding_agbm'].iloc[0]
    agbm_fileLocation="trainAgbm/"+agbm_filename
    agbm_ds=gdal.Open(agbm_fileLocation)
    agbm_data_array= agbm_ds.GetRasterBand(1).ReadAsArray()
    agbm_data_array=np.array(agbm_data_array)
    agbm_data_array=agbm_data_array.reshape(-1)
    image_numpy["y_values"]=agbm_data_array
    chip_df = pd.DataFrame(image_numpy)

    print(chip) 
    
#   Training the model
    try:
        Y_train=chip_df["y_values"]
        chip_df.drop(["y_values"],axis=1,inplace=True)
        X_train=chip_df
        #x_train,y_train,x_test,y_test=train_test_split(X_train, Y_train, test_size=0.03, random_state=101)
        adm=AdaBoostRegressor()
        grid = dict()
        grid['n_estimators'] = [10, 50, 100, 200, 500]
        grid['learning_rate'] = [0.0001,0.001, 0.01, 0.1, 1.0]
        #cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3)
        grid_search = GridSearchCV(estimator=adm, param_grid=grid, n_jobs=-1, cv=2, scoring='neg_root_mean_squared_error')
        grid_result = grid_search.fit(X_train,Y_train)
        learnrate.append(grid_result.best_params_['learning_rate'])
        n_esti.append(grid_result.best_params_['n_estimators'])
        score.append(grid_result.best_score_)
#         adm.fit(X_train,Y_train)
    except:
        print('############################')
        pass

    
    

fa341f98
95c1c3a5
176e5ab2
0331bda3
04869e3c
ff3447ad
9fec3bae
15ca148b
156a36c8
8ad42e1a
32e92a70
3fca9ea6
4a574f23
62e1f863
############################
ba514ec8
4a6680c1
70da786e
0b5de22d
6e91c55b
d7f808da
9685ddfc
7eb43782
f8843171
8be17de7
179cfc99
cb4f22a5
25420997
50b2fcde
66bc70d8
cbcc96e7
4ec27832
d5fd8f90
cbba02db
7f487274
becc8fff
462b919f
b0583a15
96b1a7fe
45a3750e
ba4c8eb5
d96589c4
6be3785d
c6b361ff
a72e67db
bd8d8000
5aff5163
1fc2a8f3
f2dd694a
f8e78ab4
69758f72
aec7576d
6bbd1e13
9f240d72
88375984
aa8a93ed
100c9e0c
d69480ac
fd61914d
59fc8d5e
256d514e
4b159655
9bd9b06a
4f52f9de
29f95745
f273e275
e52adc9f
c7a9fadb
############################
05b0083d
beca8519
c62c3224
0a9d13a4
9203898
462e7539
1f6f4cc9
53d88292
47be8408
9ad82a18
ca3dc310
7a0513e4
7451cbcf
f8b1eefd
0d0be2c8
84f04edc
704ea21b
a6e920eb
############################
32d00d9e
1834b1fc
############################
ddc53aab
c66bfd48
ce49f607
############################
c993c51d
e61ea4ab
f5d88b97
9b849ff8
5a030a2e
7

In [13]:
HPar = pd.DataFrame({'learnrate':learnrate, 'n_estimates':n_esti, 'score':score})
HPar

,learnrate,n_estimates,score
0,0.0001,10,-0.002170
1,0.0001,10,-0.001677
2,0.0100,200,-0.001842
3,0.0010,50,-0.001953
4,0.0100,200,-0.001762
...,...,...,...
225,0.0100,200,-0.001455
226,0.0010,10,-0.002036
227,0.0100,100,-0.001862
228,0.0100,500,-0.001410


In [17]:
HPar['learnrate'].value_counts()

0.0100    120
0.0001     40
0.0010     40
0.1000     29
1.0000      1
Name: learnrate, dtype: int64

In [18]:
HPar['n_estimates'].value_counts()

200    77
10     58
50     46
100    32
500    17
Name: n_estimates, dtype: int64

In [21]:
HPar[['learnrate','n_estimates']].value_counts()

learnrate  n_estimates
0.0100     200            66
0.1000     10             20
0.0100     100            18
0.0010     10             14
0.0100     50             14
           10             13
0.0010     50             12
0.0001     50             12
           10             10
0.0100     500             9
0.0001     100             8
           200             8
0.1000     50              8
0.0010     500             6
           100             5
           200             3
0.0001     500             2
0.1000     100             1
1.0000     10              1
dtype: int64